In [2]:
# import torch
import numpy as np
import data_pipes

df_path = "processed_aapl_data.csv"
data = data_pipes.process_df_3(df_path)
x, y, close = data["x"], data["y"], data["close"]
y = (y - y.min()) / (y.max() - y.min())
y = y - y.mean()
print(f"""
x shape: {x.shape}
y shape: {y.shape}     
close shape: {close.shape} 
""")


x shape: (2427, 20, 13)
y shape: (2427, 1)     
close shape: (2427, 1) 



In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

v_split = 0.5
v_cutoff = int(len(x)*(1 - v_split))
x_t = torch.tensor(x[:v_cutoff], dtype=torch.float32).to(device)
x_v = torch.tensor(x[v_cutoff:], dtype=torch.float32).to(device)
y_t = torch.tensor(y[:v_cutoff], dtype=torch.float32).to(device)
y_v = torch.tensor(y[v_cutoff:], dtype=torch.float32).to(device)
close_t = close[:v_cutoff]
close_v = close[v_cutoff:]